In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2023-09-13 10:18:02.832357: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 10:18:03.106565: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 10:18:03.108133: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 10:18:05.993383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sann-htet/anaconda3/envs/tensorflow/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = tf.keras.models.Sequential([ 
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)), 
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2), 
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2), 
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(256, activation='relu'), 
  tf.keras.layers.Dense(1, activation='sigmoid')
])

2023-09-13 10:18:10.627597: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [4]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True) 
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)

In [5]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label
     
def augment_img(image, label):
    image, label = normalize_img(image, label)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [6]:
data = data.cache()
data = data.map(augment_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = data.shuffle(1024).batch(332)
train_data = train_data.prefetch(tf.data.AUTOTUNE)

val_data = val_data.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val_data.batch(32)
val_data = val_data.cache()
val_data = val_data.prefetch(tf.data.AUTOTUNE)

In [ ]:
%time history = model.fit(train_data, epochs=10, validation_data=val_data, validation_steps=1)

Epoch 1/10


2023-09-13 10:18:15.933372: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1886907392 exceeds 10% of free system memory.
2023-09-13 10:18:17.268707: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 471726848 exceeds 10% of free system memory.
2023-09-13 10:18:17.818934: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 918296064 exceeds 10% of free system memory.


---